In [1]:
import os, sys
sys.path = ["/Users/dchaplinsky/Projects/lang-uk/lang.org.ua/languk/"] + sys.path
os.environ["DJANGO_SETTINGS_MODULE"] = "languk.settings.dev"
import django

django.setup()

In [2]:
import pymongo
from django.conf import settings
from corpus.mongodb import db
from corpus.udpipe_model import Model as UDPipeModel

In [3]:
model = UDPipeModel(settings.UDPIPE_MODEL_FILE)

In [8]:
test_doc = db.fiction.find_one({"_id": "e0555ead86335ee058bf2eef5758839f0f124ee7"})


In [9]:
test_doc["_id"]


'e0555ead86335ee058bf2eef5758839f0f124ee7'

In [20]:
import logging
logger = logging.getLogger("decompress")
logger.setLevel(logging.INFO)

from collections import OrderedDict

from corpus.ud_converter import (
    DECOMPRESS_UPOS_MAPPING,
    DECOMPRESS_FEATURES_MAPPING,
    DECOMPRESS_FEATURE_VALUES_MAPPING,
    grouper
)


def unpack_values(param_name, s):
    def _unpack_value(v):
        if param_name == "ud_postags":
            try:
                return DECOMPRESS_UPOS_MAPPING[v]
            except KeyError:
                logger.warning(
                    f"Cannot find the upos '{v}' in the mapping, skipping it for now"
                )
                return "UNK"

        elif param_name == "ud_features":
            res = []

            for c_cat, c_val in grouper(v, 2):
                try:
                    cat = DECOMPRESS_FEATURES_MAPPING[c_cat]
                except KeyError:
                    logger.warning(
                        f"Cannot find the feature '{c_cat}' in the mapping, skipping it for now"
                    )
                    cat = "UNK"

                try:
                    val = DECOMPRESS_FEATURE_VALUES_MAPPING[cat][c_val]
                except KeyError:
                    logger.warning(
                        f"Cannot find the value '{c_val}' for the feature '{cat}' in the mapping, skipping it for now"
                    )
                    
                    val = "UNK"

                res.append((cat, val))
            return OrderedDict(res)

        else:
            return v

    if param_name == "ud_postags":
        return [[_unpack_value(w) for w in l] for l in s.split("\n")]
    else:
        return [[_unpack_value(w) for w in l.split(" ")] for l in s.split("\n")]


def decompress(tokens=None, ud_lemmas=None, ud_features=None, ud_postags=None):
    params = locals()

    assert any(
        map(lambda x: x is not None, params.values())
    ), "at least one param should be not None"

    zipped = {}

    for param_name, param_value in params.items():
        if param_value is not None:
#             if param_name == "tokens":
#                 # TODO: validate if this workaround can be properly fixed
#                 param_value = param_value.strip()
            zipped[param_name] = unpack_values(param_name, param_value)
    

    sentences_length = set(map(len, zipped.values()))
    assert len(sentences_length) == 1, f"Text contains different number of sentences: {sentences_length}"

    res = []
    param_names = list(zipped.keys())
    param_values = list(zipped.values())

    for sent in zip(*param_values): 
        word_length = set(map(len, sent))

        assert len(sentences_length) == 1, f"Text contains different number of words in sentence: {sent}"

        res.append(
            [OrderedDict(zip(param_names, word_info)) for word_info in zip(*sent)]
        )


    return res


_ = decompress(
    tokens=test_doc["nlp"]["text"]["tokens"],
    ud_lemmas=test_doc["nlp"]["text"]["ud_lemmas"],
    ud_features=test_doc["nlp"]["text"]["ud_features"],
    ud_postags=test_doc["nlp"]["text"]["ud_postags"],
)


In [27]:
from ufal.udpipe import Sentence

from deepdiff import DeepDiff

def compare_article(article):
    for f in ["title", "text"]:
        if f not in article["nlp"]:
            logger.warning(f"Cannot find field {f} in the document {article['_id']}")
            continue

        if "tokens" not in article["nlp"][f]:
            logger.warning(
                f"Cannot find tokenized version of field {f} in the document {article['_id']}"
            )
            continue

        if "ud_lemmas" not in article["nlp"][f]:
            logger.warning(
                f"Cannot find lemmatized version of field {f} in the document {article['_id']}"
            )
            continue

        if "ud_features" not in article["nlp"][f]:
            logger.warning(
                f"Cannot find udpipe features of field {f} in the document {article['_id']}"
            )
            continue

        if "ud_postags" not in article["nlp"][f]:
            logger.warning(
                f"Cannot find udpipe postags of field {f} in the document {article['_id']}"
            )
            continue
            

        decompressed_result = decompress(
            tokens=article["nlp"][f]["tokens"],
            ud_lemmas=article["nlp"][f]["ud_lemmas"],
            ud_features=article["nlp"][f]["ud_features"],
            ud_postags=article["nlp"][f]["ud_postags"],
        )
        udpipe_res = []

        for s in article["nlp"][f]["tokens"].split("\n"):
            tok_sent = Sentence()
            for w in s.split(" "):
                tok_sent.addWord(w)

            model.tag(tok_sent)

            udpipe_res.append(
                [
                    {
                        "tokens": w.form,
                        "ud_lemmas": w.lemma,
                        "ud_postags": w.upostag,
                        "ud_features": OrderedDict(
                            (f.split("=") for f in w.feats.split("|"))
                        ) if w.feats else OrderedDict(),
                    }
                    for w in tok_sent.words[1:]
                ]
            )

        comparison = udpipe_res == decompressed_result
        if not comparison:
            print(f"Comparing {f} for the {article['_id']}: {comparison}")
        
        if not comparison:
            with open("udpipe_res.json", "w") as fp_out:
                json.dump(udpipe_res, fp_out, indent=4, ensure_ascii=False, sort_keys=True)
            with open("decompressed_result.json", "w") as fp_out:
                json.dump(decompressed_result, fp_out, indent=4, ensure_ascii=False, sort_keys=True)

            return False

    return True


compare_article(test_doc)

Cannot find lemmatized version of field title in the document d6c6f1f0fc7c6bce04bd2a3b84d92a2d5f9bddb0
Cannot find lemmatized version of field text in the document d6c6f1f0fc7c6bce04bd2a3b84d92a2d5f9bddb0


True

In [28]:
from tqdm.notebook import tqdm
for test_doc in tqdm(db.fiction.find()):
    if not compare_article(test_doc):
        print(f"Comparison failed on {test_doc['_id']}")
        break

        

0it [00:00, ?it/s]

Cannot find field title in the document 3eafcb11ef77e173a6c2cebdb49ca58baad12c71
Cannot find field text in the document 3eafcb11ef77e173a6c2cebdb49ca58baad12c71
Cannot find lemmatized version of field title in the document d6c6f1f0fc7c6bce04bd2a3b84d92a2d5f9bddb0
Cannot find lemmatized version of field text in the document d6c6f1f0fc7c6bce04bd2a3b84d92a2d5f9bddb0


KeyError: 'nlp'

In [6]:
s = """Звернімося просто до його творчої біографії яка починається ще на порозі юності збіркою 1910 року На білих островах і поки що доходить до книжок віршів виданих у 1957 1959 роках
За підрахунком одного з критиків М. Рильського у цілому це складає більше 25 збірок оригінальних поезій і понад 250 тисяч рядків поетичних перекладів а до того слід додати ще численні статті і дослідження з історії літератури народної творчості театру багато публіцистичних виступів
Початкове формування таланту М. Рильського припадає на роки 1907 1917"""

s = "так-сяк понад 250 тисяч"

In [15]:
for ss in s.split("\n"):
    tokenized = model.tokenize(ss)
    for tok_sent in tokenized:
        sent_lemmas = []
        sent_postags = []
        sent_features = []

        model.tag(tok_sent)

        for w in tok_sent.words[1:]:
            print(w.form, w.lemma, w.upostag)

так так ADV
- - PUNCT
сяк сяк ADV
понад 250 понад 250 NUM
тисяч тисяча NOUN


In [14]:
model.tokenizer = model.model.newTokenizer(model.model.TOKENIZER_PRESEGMENTED)

In [26]:
tokenizer = model.model.newTokenizer(model.model.TOKENIZER_NORMALIZED_SPACES)

In [30]:
import ufal.udpipe


for tok in [model.model.TOKENIZER_PRESEGMENTED, model.model.TOKENIZER_NORMALIZED_SPACES, model.model.TOKENIZER_RANGES]:
    tokenizer = model.model.newTokenizer(tok)
    tokenizer.setText(s)

    error = ufal.udpipe.ProcessingError()
    sentences = []

    sentence = ufal.udpipe.Sentence()
    while tokenizer.nextSentence(sentence, error):
        sentences.append(sentence)
        sentence = ufal.udpipe.Sentence()

    print(sentences[0].words[4].form)
    

понад 250
понад 250
понад 250
